In [47]:
:dep graph-ds = { path = "." }

In [48]:
use graph_ds::{Graph, Node, Edge};

In [49]:
let mut graph : Graph<u64> = Graph::new();

// add a few nodes
let nodes : Vec<Node<u64>> = (0 .. 10).map(|i| {
    Node {
        id: i as u64,
        layer: None,
    }
}).collect::<_>();

// make a few edges
graph.build_and_add_egde(&nodes[0], &nodes[1], Some(1.0), None);
graph.build_and_add_egde(&nodes[1], &nodes[0], Some(1.0), None);
graph.build_and_add_egde(&nodes[1], &nodes[2], Some(1.0), None);

In [50]:
graph.edges.get(&0_u64)

Some([Edge { from: (Weak), to: (Weak), weight: Some(1.0), capacity: None }])

In [51]:
graph.bfs(0, None)

start node: 0
nodes: Some([Edge { from: (Weak), to: (Weak), weight: Some(1.0), capacity: None }])


Ok((None, {1: 1.0, 2: 2.0, 0: 0.0}))

In [6]:
graph.edges

{1: [Edge { from: (Weak), to: (Weak), weight: Some(1.0), capacity: None }, Edge { from: (Weak), to: (Weak), weight: Some(1.0), capacity: None }]}

In [7]:
println!("{}", graph.edges.get(&1).unwrap().first().unwrap())

Edge { from: Some(Node { id: 1, layer: None }), to: None, weight: Some(1.0), capacity: None }


()

In [8]:
graph.nodes

[None, Some(Node { id: 1, layer: None }), Some(Node { id: 2, layer: None })]